In [1]:
from utilities.utilities import load_data, get_records_by_region, create_column, finalize_dataframe
# settings
region_column_name = 'Region'
column_name_to_results_global = 'Global #'

source_df = load_data('security_headers_checker')

In [2]:
# sanity dataset

headers = [
            'strict-transport-security',
            'x-frame-options',
            'x-content-type-options',
            'content-security-policy',
            'x-permitted-cross-domain-policies',
            'referrer-policy',
            'clear-site-data',
            'cross-origin-embedder-policy',
            'cross-origin-opener-policy',
            'cross-origin-resource-policy',
            'cache-control'
        ]

# Records that are null suffered an error during data collection and are therefore treated as not having security headers.
for header in headers:
    source_df.loc[source_df[header].isna(), header] = False



In [3]:
from utilities.utilities import get_extreme_values

# Analysis of HEIs with at least one OWASP Recommended Security Headers by region

# settings
column_to_sort = 'Without Security Headers (Public) %'
sort_ascending = True
columns_to_display = [region_column_name.title(), column_name_to_results_global]
analysis_df = get_records_by_region(source_df)

# create columns
# Column creation with distribution of records without any security headers by region
only_public = 'category == "Public"'
only_headers_false = ' & '.join([f"`{h}` == False" for h in headers])
criteria = f"{only_public} & ({only_headers_false})"
create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='Without Security Headers (Public)', criteria=criteria, columns_to_display=columns_to_display)
only_private = 'category == "Private"'
criteria = f"{only_private} & ({only_headers_false})"
create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='Without Security Headers (Private)', criteria=criteria, columns_to_display=columns_to_display)

# Column creation with distribution of records with at least one security header by region
all_true_filter = ' & '.join([f"`{h}` == True" for h in headers])
at_least_one_filter = ' | '.join([f"`{h}` == True" for h in headers])
composite_filter = f"({at_least_one_filter}) & ~({all_true_filter})"
criteria = f"{only_public} & ({composite_filter})"
create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='With At Least One Security Header (Public)', criteria=criteria, columns_to_display=columns_to_display)
criteria = f"{only_private} & ({composite_filter})"
create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='With At Least One Security Header (Private)', criteria=criteria, columns_to_display=columns_to_display)

# Column creation with distribution of records with all security header by region
criteria = f"{only_public} & ({all_true_filter})"
create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='With All Security Headers (Public)', criteria=criteria, columns_to_display=columns_to_display)
criteria = f"{only_private} & ({all_true_filter})"
create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='With All Security Headers (Private)', criteria=criteria, columns_to_display=columns_to_display)

# Finalize dataframe
analysis_df = finalize_dataframe(dataframe=analysis_df, column_to_sort=column_to_sort, ascending=sort_ascending, columns_to_display=columns_to_display)
display(analysis_df)
#analysis_df.to_csv(f'tables/test.csv', index=False)

# report
report_results = get_extreme_values(analysis_df)
report_figure = """
\\begin{figure}[htbp]
    \centering
    \includegraphics[width=0.48\\textwidth]{charts/Security_headers_by_region.pdf}
    \caption{Distribution of the use Security Headers.}\label{fig:security-headers}
\end{figure}
"""
hei_public_without_security_headers = format(report_results.get("Total").get("Without Security Headers (Public) %"), ".2f")
hei_public_with_security_headers = format(report_results.get("Total").get("With At Least One Security Header (Public) %"), ".2f")
hei_public_with_all_security_headers = format(report_results.get("Total").get("With All Security Headers (Public) %"), ".2f")
hei_private_without_security_headers = format(report_results.get("Total").get("Without Security Headers (Private) %"), ".2f")
hei_private_with_security_headers = format(report_results.get("Total").get("With At Least One Security Header (Private) %"), ".2f")
hei_private_with_all_security_headers = format(report_results.get("Total").get("With All Security Headers (Private) %"), ".2f")
report = f'{report_figure}\n\n'
report += f'The data presented in Figure~\\ref{{fig:security-headers}} show the use of security headers at \glspl{{hei}} in the United States of America. '
report += f'According to the data, {hei_public_without_security_headers}\% of the public institutions analyzed have not implemented any security headers in their websites. '
report += f'On the other hand, {hei_private_without_security_headers}\% of the private institutions analyzed have not implemented any security headers in their websites.\n\n'
report += f'On a positive note, {hei_public_with_security_headers}\% of the public institutions analyzed have implemented at least one security header in their websites. '
report += f'Moreover, {hei_private_with_security_headers}\% of the private institutions analyzed have implemented at least one security header in their websites.\n\n'
report += f'Finally, {hei_public_with_all_security_headers}\% of the public institutions analyzed have implemented all the security headers in their websites. '
report += f'On the other hand, {hei_private_with_all_security_headers}\% of the private institutions analyzed have implemented all the security headers in their websites.\n\n'
report += f'The regions that stand out in the use of security headers by private institutions are {report_results.get("Without Security Headers (Private) %").get("top_regions")[0][0]} ({format(report_results.get("Without Security Headers (Private) %").get("top_regions")[0][1], ".2f")}\%), {report_results.get("Without Security Headers (Private) %").get("top_regions")[1][0]} ({format(report_results.get("Without Security Headers (Private) %").get("top_regions")[1][1], ".2f")}\%), and {report_results.get("Without Security Headers (Private) %").get("top_regions")[2][0]} ({format(report_results.get("Without Security Headers (Private) %").get("top_regions")[2][1], ".2f")}\%). \n\n'
report += f'The regions that stand out in the use of security headers by public institutions are {report_results.get("Without Security Headers (Public) %").get("top_regions")[0][0]} ({format(report_results.get("Without Security Headers (Public) %").get("top_regions")[0][1], ".2f")}\%), {report_results.get("Without Security Headers (Public) %").get("top_regions")[1][0]} ({format(report_results.get("Without Security Headers (Public) %").get("top_regions")[1][1], ".2f")}\%), and {report_results.get("Without Security Headers (Public) %").get("top_regions")[2][0]} ({format(report_results.get("Without Security Headers (Public) %").get("top_regions")[2][1], ".2f")}\%). \n\n'
report += f'The regions with the lowest use of security headers by private institutions are {report_results.get("With All Security Headers (Private) %").get("bottom_regions")[0][0]} ({format(report_results.get("With All Security Headers (Private) %").get("bottom_regions")[0][1], ".2f")}\%), {report_results.get("With All Security Headers (Private) %").get("bottom_regions")[1][0]} ({format(report_results.get("With All Security Headers (Private) %").get("bottom_regions")[1][1], ".2f")}\%), and {report_results.get("With All Security Headers (Private) %").get("bottom_regions")[2][0]} ({format(report_results.get("With All Security Headers (Private) %").get("bottom_regions")[2][1], ".2f")}\%). \n\n'
report += f'The regions with the lowest use of security headers by public institutions are {report_results.get("With All Security Headers (Public) %").get("bottom_regions")[0][0]} ({format(report_results.get("With All Security Headers (Public) %").get("bottom_regions")[0][1], ".2f")}\%), {report_results.get("With All Security Headers (Public) %").get("bottom_regions")[1][0]} ({format(report_results.get("With All Security Headers (Public) %").get("bottom_regions")[1][1], ".2f")}\%), and {report_results.get("With All Security Headers (Public) %").get("bottom_regions")[2][0]} ({format(report_results.get("With All Security Headers (Public) %").get("bottom_regions")[2][1], ".2f")}\%). \n\n'


print(report)

,Region,Global #,Without Security Headers (Public) #,Without Security Headers (Public) %,Without Security Headers (Private) #,Without Security Headers (Private) %,With At Least One Security Header (Public) #,With At Least One Security Header (Public) %,With At Least One Security Header (Private) #,With At Least One Security Header (Private) %,With All Security Headers (Public) #,With All Security Headers (Public) %,With All Security Headers (Private) #,With All Security Headers (Private) %
0,California,2,0,0.000000,0,0.000000,0,0.0,2,100.000000,0,0.0,0,0.0
1,Florida,1,0,0.000000,0,0.000000,0,0.0,1,100.000000,0,0.0,0,0.0
2,Georgia,1,0,0.000000,1,100.000000,0,0.0,0,0.000000,0,0.0,0,0.0
3,Illinois,1,0,0.000000,0,0.000000,0,0.0,1,100.000000,0,0.0,0,0.0
4,Michigan,1,0,0.000000,0,0.000000,0,0.0,1,100.000000,0,0.0,0,0.0
5,Minnesota,1,0,0.000000,0,0.000000,0,0.0,1,100.000000,0,0.0,0,0.0
6,New York,1,0,0.000000,0,0.000000,0,0.0,1,100.000000,0,0.0,0,0.0
7,Texas,2,0,0.000000,2,100.000000,0,0.0,0,0.000000,0,0.0,0,0.0
8,Colorado,1,1,100.000000,0,0.000000,0,0.0,0,0.000000,0,0.0,0,0.0
9,Total,11,1,9.090909,3,27.272727,0,0.0,7,63.636364,0,0.0,0,0.0



\begin{figure}[htbp]
    \centering
    \includegraphics[width=0.48\textwidth]{charts/Security_headers_by_region.pdf}
    \caption{Distribution of the use Security Headers.}\label{fig:security-headers}
\end{figure}


The data presented in Figure~\ref{fig:security-headers} show the use of security headers at \glspl{hei} in the United States of America. According to the data, 9.09\% of the public institutions analyzed have not implemented any security headers in their websites. On the other hand, 27.27\% of the private institutions analyzed have not implemented any security headers in their websites.

On a positive note, 0.00\% of the public institutions analyzed have implemented at least one security header in their websites. Moreover, 63.64\% of the private institutions analyzed have implemented at least one security header in their websites.

Finally, 0.00\% of the public institutions analyzed have implemented all the security headers in their websites. On the other hand, 0.00\% of t

In [4]:
\begin{figure}[htbp]
    \centering
    \includegraphics[width=0.48\textwidth]{charts/Security_headers_by_region.pdf}
    \caption{Distribution of the use Security Headers.}\label{fig:security-headers}
\end{figure}


The data presented in Figure~\ref{fig:security-headers} show the use of security headers at \glspl{hei} in the United States of America. According to the data, 9.09\% of the public institutions analyzed have not implemented any security headers in their websites. On the other hand, 27.27\% of the private institutions analyzed have not implemented any security headers in their websites.

On a positive note, 0.00\% of the public institutions analyzed have implemented at least one security header in their websites. Moreover, 63.64\% of the private institutions analyzed have implemented at least one security header in their websites.

Finally, 0.00\% of the public institutions analyzed have implemented all the security headers in their websites. On the other hand, 0.00\% of the private institutions analyzed have implemented all the security headers in their websites.

The regions that stand out in the use of security headers by private institutions are Georgia (100.00\%), Texas (100.00\%), and California (0.00\%).

The regions that stand out in the use of security headers by public institutions are Colorado (100.00\%), California (0.00\%), and Florida (0.00\%).

The regions with the lowest use of security headers by private institutions are California (0.00\%), Florida (0.00\%), and Georgia (0.00\%).

The regions with the lowest use of security headers by public institutions are California (0.00\%), Florida (0.00\%), and Georgia (0.00\%).


{'Without Security Headers (Public) %': {'top_regions': [('Colorado', 100.0),
   ('Total', 9.09090909),
   ('California', 0.0)],
  'bottom_regions': [('California', 0.0), ('Florida', 0.0), ('Georgia', 0.0)]},
 'Without Security Headers (Private) %': {'top_regions': [('Georgia', 100.0),
   ('Texas', 100.0),
   ('Total', 27.27272727)],
  'bottom_regions': [('California', 0.0),
   ('Florida', 0.0),
   ('Illinois', 0.0)]},
 'Total': {'Without Security Headers (Public) %': 1,
  'Without Security Headers (Private) %': 3},
 'With At Least One Security Header (Public)': 4,
 'With At Least One Security Header (Private)': 11,
 'With All Security Headers (Public)': 0,
 'With All Security Headers (Private)': 0}